# Import the required libraries <a class="anchor" id="import_libraries"></a>

In [ ]:
%matplotlib widget
import hyperspy.api as hs
import numpy as np
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import atomap.api as am
import tkinter as tk
from tkinter.filedialog import askopenfilename
import os
hs.preferences.GUIs.warn_if_guis_are_missing = False
hs.preferences.save()
plt.rcParams['figure.figsize'] = (8,8)

# Open files<a class="anchor" id="open_files"></a>
- Heterostructure HAADF image .tif (16 bits grayscale)
<br>

In [ ]:
root = tk.Tk()
root.attributes('-topmost',True)
root.iconify()   
file_path = askopenfilename(parent=root)
root.destroy()
#s=hs.load(file_path)
s=hs.signals.Signal2D(plt.imread(file_path))
path = os.path.splitext(file_path)[0]
if not (os.path.exists(path)):
    os.mkdir(path)

## Axes scaling (pixel distance) <a class="anchor" id="pixel_distance"></a>

> **`pixel_size_nm`**: pixel size in nanometers.
<br>
>An aspect ratio of 1 is assumed.

In [ ]:
#pixel_size_pm=float(input('pixel_size_pm: '))
pixel_size_pm=6.652

s.axes_manager[0].name = 'X'
s.axes_manager[1].name = 'Y'
s.axes_manager[0].scale = pixel_size_pm/1000
s.axes_manager[1].scale = pixel_size_pm/1000
s.axes_manager[0].units = 'nm'
s.axes_manager[1].units = 'nm'
s.metadata.General.title = ''
s.plot(colorbar=False)
ax=plt.gca()
norm = mpl.colors.Normalize(vmin=np.min(s.data), vmax=np.max(s.data))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="10%", pad=0.15)
plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='Greys_r'),ax=ax, pad=.05, fraction=.1,cax=cax)
plt.tight_layout()

## Intensity scaling (pixel intensity) <a class="anchor" id="pixel_intensity"></a>

- Detector image .tif (16 bits grayscale) - for the normalization to the impinging beam.

> **`inner_angle`**: inner collection angle in mrads.
<br>
> **`outer_angle`**: outer collection angle in mrads.
<br>

In [ ]:
inner_angle=130
outer_angle=200

root = tk.Tk()
root.attributes('-topmost',True)
root.iconify()   
det_image = askopenfilename(parent=root)
root.destroy()
#det_image=hs.load(det_image)
det_image=hs.signals.Signal2D(plt.imread(det_image))
s_normalised = am.quant.detector_normalisation(s, det_image, inner_angle=inner_angle, outer_angle=outer_angle)
s_normalised.plot(colorbar=False)
ax=plt.gca()
norm = mpl.colors.Normalize(vmin=np.min(s_normalised.data), vmax=np.max(s_normalised.data))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="10%", pad=0.15)
plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='Greys_r'),ax=ax, pad=.05, fraction=.1,cax=cax)
plt.tight_layout()

In [ ]:
plt.savefig(path+'\\normalized_image.png',dpi=512,transparent=True,bbox_inches='tight')

# Denoising <a class="anchor" id="denoising"></a>
> **`pixels_cropped`**: pixels to be cropped from the outer frame to remove side effects of the drift correction and RL denoising.

In [ ]:
from skimage.restoration import estimate_sigma

pixels_cropped=32
original_imag=s_normalised.data
sigma_est = np.mean(estimate_sigma(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]))
print(f'estimated noise standard deviation = {sigma_est}')
fig, ax = plt.subplots(figsize=(12,8))
ax.set_title('Original Image')
ax.imshow(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax.axis('off')


In [ ]:
plt.savefig(path+'\\original'+'_'+str(round(sigma_est,12))+'.png',dpi=512,transparent=True,bbox_inches='tight')

- PCA (Principal Component Analysis):

> **`n_components`**: number of components in PCA.

In [ ]:
from sklearn import decomposition

n_components=8
pca = decomposition.PCA(n_components=n_components)
pca.fit(original_imag)
pcaFaces = pca.transform(original_imag)
pca_imag = pca.inverse_transform(pcaFaces)

In [ ]:
sigma_est = np.mean(estimate_sigma(pca_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]))
print(f'estimated noise standard deviation = {sigma_est}')
fig, ax = plt.subplots(figsize=(12,8), nrows=1, ncols=3)

ax[0].set_title('Original Image')
ax[0].imshow(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[0].axis('off')


ax[1].set_title('PCA Image '+str(n_components)+' components')
ax[1].imshow(pca_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[1].axis('off')

ax[2].set_title('Residuals')
ax[2].imshow(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]-pca_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[2].axis('off')

fig.tight_layout()
plt.show()

In [ ]:
plt.savefig(path+'\\PCA'+str(n_components)+'_'+str(round(sigma_est,12))+'.png',dpi=512,transparent=True,bbox_inches='tight')

- NL means (Non-local means):

> **`h`**: cut-off distance (in gray levels).
<br>
> **`patch_size`**: size of patches used for denoising.
<br>
> **`patch_distance`**: maximal distance in pixels where to search patches used for denoising.

In [ ]:
from skimage.restoration import denoise_nl_means

sigma_est = np.mean(estimate_sigma(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]))
h=5
patch_size=5
patch_distance=6
nlm_imag = denoise_nl_means(original_imag, h=h*sigma_est, fast_mode=True,patch_size=patch_size,patch_distance=patch_distance)

In [ ]:
sigma_est = np.mean(estimate_sigma(nlm_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]))
print(f'estimated noise standard deviation = {sigma_est}')
fig, ax = plt.subplots(figsize=(12,8), nrows=1, ncols=3)

ax[0].set_title('Original Image')
ax[0].imshow(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[0].axis('off')


ax[1].set_title(f'Non-local means denoising {h}$\\sigma$ patch {patch_size}-{patch_distance}')
ax[1].imshow(nlm_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[1].axis('off')

ax[2].set_title('Residuals')
ax[2].imshow(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]-nlm_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[2].axis('off')
fig.tight_layout()
plt.show()

In [ ]:
plt.savefig(path+'\\NLM_h'+str(h)+'_p'+str(patch_size)+'-'+str(patch_distance)+'_'+str(round(sigma_est,12))+'.png',dpi=512,transparent=True,bbox_inches='tight')

- RL deconvolution (Richardson-Lucy deconvolution):

> **`probe_resolution`**: Probe resolution of the HAADF detector.
<br>
> **`iter`**: Number of iterations.

In [ ]:
from scipy import signal
from skimage.restoration import richardson_lucy

def gkern(kernlen, std):
    gkern1d = signal.gaussian(kernlen, std=std).reshape(kernlen, 1)
    gkern2d = np.outer(gkern1d, gkern1d)
    return gkern2d
pixel_size=pixel_size_pm/1000
probe_resolution=0.065
width_pix=round(probe_resolution/pixel_size)
gaussian_probe=gkern(4*width_pix,std=width_pix/2.348)
plt.figure(figsize=(4,4))
plt.imshow(gaussian_probe,cmap='gray')
iter=5
rl_imag = richardson_lucy(original_imag, gaussian_probe, iter)

In [ ]:
sigma_est = np.mean(estimate_sigma(rl_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]))
print(f'estimated noise standard deviation = {sigma_est}')
fig, ax = plt.subplots(figsize=(12,8), nrows=1, ncols=3)

ax[0].set_title('Original Image')
ax[0].imshow(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[0].axis('off')


ax[1].set_title(f'Deconvolved {iter} iter')
ax[1].imshow(rl_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[1].axis('off')

ax[2].set_title('Residuals')
ax[2].imshow(original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]-rl_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped], cmap='gray')
ax[2].axis('off')
fig.tight_layout()
plt.show()

In [ ]:
plt.savefig(path+'\\RL'+str(iter)+'iter_'+str(round(sigma_est,12))+'.png',dpi=512,transparent=True,bbox_inches='tight')

# Get the sublattices <a class="anchor" id="sublattices"></a>

> **`optimal_separation`**: separation in pixels to recognize the  brighter peaks (atom columns).

In [ ]:
s_normalised.data=original_imag[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]
#optimal_separation=float(input('optimal_separation: '))
optimal_separation=9

atom_positions = am.get_atom_positions(s_normalised, optimal_separation,pca=True,subtract_background=True, normalize_intensity=True)
sublattice = am.Sublattice(atom_positions, s_normalised)
sublattice.get_atom_list_on_image(markersize=5).plot()
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig = plt.gcf()
fig.set_size_inches((8,8))

In [ ]:
import atomap.initial_position_finding as ipf
dumbbell_vector = ipf.find_dumbbell_vector(atom_positions)

> **`optimal_separation_d`**: separation in pixels to recognize the  dumbbell positions.

In [ ]:
#optimal_separation=float(input('optimal_separation: '))
optimal_separation_d=24
dumbbell_positions = am.get_atom_positions(s_normalised, optimal_separation_d,pca=True,subtract_background=True, normalize_intensity=True)
sublattice = am.Sublattice(dumbbell_positions, s_normalised)
sublattice.get_atom_list_on_image(markersize=5).plot()
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig = plt.gcf()
fig.set_size_inches((8,8))

- Once we have found the optimal separation we can add or remove the missing dumbbell positions with the **`add_atoms_with_gui`** function.

In [ ]:
dumbbell_positions = am.add_atoms_with_gui(s_normalised, dumbbell_positions,distance_threshold=10)
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig = plt.gcf()
fig.set_size_inches((16,16))

- Dumbbell recognition:

In [ ]:
dumbbell_positions = np.asarray(dumbbell_positions)
dumbbell_lattice = ipf.make_atom_lattice_dumbbell_structure(s_normalised, dumbbell_positions, dumbbell_vector)
dumbbell_lattice.pixel_size=s_normalised.axes_manager[0].scale
dumbbell_lattice.sublattice_list[0].pixel_size=s_normalised.axes_manager[0].scale
dumbbell_lattice.sublattice_list[1].pixel_size=s_normalised.axes_manager[0].scale
dumbbell_lattice.units=s_normalised.axes_manager[0].units
dumbbell_lattice.sublattice_list[0].units=s_normalised.axes_manager[0].units
dumbbell_lattice.sublattice_list[1].units=s_normalised.axes_manager[0].units


In [ ]:
dumbbell_lattice.plot(markersize=5)
ax = plt.gca()
fig = plt.gcf()
ax.set_title('')
ax.images[-1].colorbar.remove()
fig.set_size_inches((s_normalised.data.shape[1]/512,s_normalised.data.shape[0]/512))
fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
ax.axis('off')

In [ ]:
plt.savefig(path+'\\dumbbells.png',dpi=512,transparent=True)
dumbbell_lattice.save(path+'\\data.hdf5', overwrite=True)

# Filtering in frequency <a class="anchor" id="frequency"></a>

> **`text`**; selected denoised image:
- **`original_imag`** for original.
- **`pca_imag`** for PCA.
- **`nlm_imag`** for NL-means.
- **`rl_imag`** for RL deconvolution.

> **High-pass filter**: 2nd order butterworth filter with cutoff frequency ratio of 0.005.
<br>
> **Low-pass filter**: 4th order butterworth filter  with cutoff frequency ratio of 0.08.

In [ ]:
from scipy import fft
from skimage.filters import butterworth

text='pca_imag'

image=eval(text)[pixels_cropped:-pixels_cropped,pixels_cropped:-pixels_cropped]
spectrum=fft.fftshift(fft.fft2(image))
high_pass=np.mean(np.abs(fft.ifft2((fft.fft2(image)==fft.fft2(image)[0,0])*fft.fft2(image))))+butterworth(image,0.005,True,order=2)
band_pass=butterworth(high_pass,0.08,False,order=4)
np.save(path+'\\high_pass_'+text+'.npy',high_pass)
np.save(path+'\\band_pass_'+text+'.npy',band_pass)
fig,axs= plt.subplots(2,2,figsize=(16,16))
img1=axs[0,0].imshow(image,cmap='gray')
fig.colorbar(img1,shrink=0.4)
img2=axs[0,1].imshow(high_pass,cmap='gray')
fig.colorbar(img2,shrink=0.4)
spectrum2=fft.fftshift(fft.fft2(high_pass))
img3=axs[1,0].imshow(np.log(np.abs(spectrum)),cmap='gray',vmin=-7.5,vmax=15)
fig.colorbar(img3,shrink=0.4)
img4=axs[1,1].imshow(np.log(np.abs(spectrum2)),cmap='gray',vmin=-7.5,vmax=15)
fig.colorbar(img4,shrink=0.4)
axs[0,0].axis('off')
axs[0,1].axis('off')
axs[1,0].axis('off')
axs[1,1].axis('off')
plt.tight_layout()

In [ ]:
plt.savefig(path+'\\high_pass_'+text+'.png',dpi=600,transparent=True,bbox_inches='tight')

fig,axs= plt.subplots(2,2,figsize=(16,16))
img1=axs[0,0].imshow(image,cmap='gray')
fig.colorbar(img1,shrink=0.4)
img2=axs[0,1].imshow(band_pass,cmap='gray')
fig.colorbar(img2,shrink=0.4)
spectrum2=fft.fftshift(fft.fft2(band_pass))
img3=axs[1,0].imshow(np.log(np.abs(spectrum)),cmap='gray',vmin=-7.5,vmax=15)
fig.colorbar(img3,shrink=0.4)
img4=axs[1,1].imshow(np.log(np.abs(spectrum2)),cmap='gray',vmin=-7.5,vmax=15)
fig.colorbar(img4,shrink=0.4)
axs[0,0].axis('off')
axs[0,1].axis('off')
axs[1,0].axis('off')
axs[1,1].axis('off')
plt.tight_layout()

In [ ]:
plt.savefig(path+'\\band_pass_'+text+'.png',dpi=600,transparent=True,bbox_inches='tight')